In [1]:
import numpy as np
from pymatching import Matching
import matplotlib.pyplot as plt
from scipy import sparse, linalg
import CompassCodes as cc
import stim 
import pandas as pd
from compass_code_correlated_error import depolarizing_err

In [50]:
df_test = pd.read_csv('x_corr_err_data.csv')

In [51]:
df_test = df_test[df_test["error_type"] != "CORR_XZ"]
df_test

,d,num_shots,p,l,eta,error_type,num_log_errors,time_stamp
0,11.0,10000.0,0.010000,6.0,5.0,X,0.0000,2024-12-05 10:37:53.398465
1,11.0,10000.0,0.010000,6.0,5.0,Z,0.0000,2024-12-05 10:37:53.402317
2,11.0,10000.0,0.010000,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:53.403587
3,11.0,10000.0,0.010000,6.0,5.0,TOTAL,0.0000,2024-12-05 10:37:53.404731
4,11.0,10000.0,0.022564,6.0,5.0,X,0.0000,2024-12-05 10:37:56.649020
...,...,...,...,...,...,...,...,...
2399993,19.0,10000.0,0.487436,5.0,10.0,Z,0.5002,2025-03-18 19:00:21.853417
2399995,19.0,10000.0,0.487436,5.0,10.0,TOTAL,0.5302,2025-03-18 19:00:21.855776
2399996,19.0,10000.0,0.500000,5.0,10.0,X,0.0339,2025-03-18 19:00:33.072727
2399997,19.0,10000.0,0.500000,5.0,10.0,Z,0.5009,2025-03-18 19:00:33.074595


In [52]:
df_test.to_csv('x_corr_err_data.csv', index=False)

In [25]:
df_test.loc[df_test['error_type'] == 'x', 'error_type'] = 'X'
df_test.loc[df_test['error_type'] == 'z', 'error_type'] = 'Z'
df_test.loc[df_test['error_type'] == 'total', 'error_type'] = 'TOTAL'
df_test.loc[df_test['error_type'] == 'CORR_ZX', 'error_type'] = 'CORR_XZ'
df_test

,d,num_shots,p,l,eta,error_type,num_log_errors,time_stamp
0,11.0,1000.0,0.010000,6.0,5.89,X,0.0000,2024-08-08 16:11:22.527852
1,11.0,1000.0,0.010000,6.0,5.89,Z,0.0000,2024-08-08 16:11:22.530430
2,11.0,1000.0,0.010000,6.0,5.89,CORR_XZ,0.0000,2024-08-08 16:11:22.531261
3,11.0,1000.0,0.010000,6.0,5.89,TOTAL,0.0000,2024-08-08 16:11:22.532018
4,11.0,1000.0,0.064444,6.0,5.89,X,0.0000,2024-08-08 16:11:22.997865
...,...,...,...,...,...,...,...,...
1933315,19.0,10000.0,0.487436,4.0,10.00,TOTAL,0.5120,2025-03-18 17:13:18.146044
1933316,19.0,10000.0,0.500000,4.0,10.00,X,0.0177,2025-03-18 17:13:29.735405
1933317,19.0,10000.0,0.500000,4.0,10.00,Z,0.4971,2025-03-18 17:13:29.738585
1933318,19.0,10000.0,0.500000,4.0,10.00,CORR_XZ,0.5188,2025-03-18 17:13:29.740307


In [36]:
df_xz.to_csv('corr_err_data.csv', index=False)

In [45]:
df = pd.read_csv('x_corr_err_data.csv')
df.loc[df['d'] == 11]['num_log_errors']


0          0.0000
1          0.0000
2          0.0000
3          0.0000
4          0.0000
            ...  
2399355    0.5407
2399356    0.0341
2399357    0.5006
2399358    0.5360
2399359    0.5347
Name: num_log_errors, Length: 480000, dtype: float64

In [94]:
df_zx = df[df['error_type'] == 'CORR_ZX']
df_z = df[df['error_type'] == 'Z']

df_zx

,d,num_shots,p,l,eta,error_type,num_log_errors,time_stamp
2,11.0,10000.0,0.010000,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:53.403587
6,11.0,10000.0,0.022564,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:56.653524
10,11.0,10000.0,0.035128,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:59.932352
14,11.0,10000.0,0.047692,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:38:03.298141
18,11.0,10000.0,0.060256,6.0,5.0,CORR_ZX,0.0002,2024-12-05 10:38:06.584896
...,...,...,...,...,...,...,...,...
2319982,19.0,10000.0,0.449744,6.0,0.5,CORR_ZX,0.5000,2025-03-07 13:58:46.447681
2319986,19.0,10000.0,0.462308,6.0,0.5,CORR_ZX,0.5007,2025-03-07 13:58:54.341651
2319990,19.0,10000.0,0.474872,6.0,0.5,CORR_ZX,0.5062,2025-03-07 13:59:02.711583
2319994,19.0,10000.0,0.487436,6.0,0.5,CORR_ZX,0.4961,2025-03-07 13:59:10.643044


In [95]:
df_z.reset_index(inplace=True)

In [96]:
df_zx.reset_index(inplace=True)

In [104]:
df_z

,index,d,num_shots,p,l,eta,error_type,num_log_errors,time_stamp
0,1,11.0,10000.0,0.010000,6.0,5.0,Z,0.0000,2024-12-05 10:37:53.402317
1,5,11.0,10000.0,0.022564,6.0,5.0,Z,0.0000,2024-12-05 10:37:56.652305
2,9,11.0,10000.0,0.035128,6.0,5.0,Z,0.0000,2024-12-05 10:37:59.931137
3,13,11.0,10000.0,0.047692,6.0,5.0,Z,0.0001,2024-12-05 10:38:03.296961
4,17,11.0,10000.0,0.060256,6.0,5.0,Z,0.0009,2024-12-05 10:38:06.583673
...,...,...,...,...,...,...,...,...,...
579995,2319981,19.0,10000.0,0.449744,6.0,0.5,Z,0.4982,2025-03-07 13:58:46.446507
579996,2319985,19.0,10000.0,0.462308,6.0,0.5,Z,0.5012,2025-03-07 13:58:54.340467
579997,2319989,19.0,10000.0,0.474872,6.0,0.5,Z,0.5023,2025-03-07 13:59:02.710089
579998,2319993,19.0,10000.0,0.487436,6.0,0.5,Z,0.5046,2025-03-07 13:59:10.641895


In [98]:
df_zx['num_log_errors'] = df_zx['num_log_errors'] + df_z['num_log_errors']
df_zx

<ipython-input-98-7c1ec53cbc45>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_zx['num_log_errors'] = df_zx['num_log_errors'] + df_z['num_log_errors']


,index,d,num_shots,p,l,eta,error_type,num_log_errors,time_stamp
0,2,11.0,10000.0,0.010000,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:53.403587
1,6,11.0,10000.0,0.022564,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:56.653524
2,10,11.0,10000.0,0.035128,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:59.932352
3,14,11.0,10000.0,0.047692,6.0,5.0,CORR_ZX,0.0001,2024-12-05 10:38:03.298141
4,18,11.0,10000.0,0.060256,6.0,5.0,CORR_ZX,0.0011,2024-12-05 10:38:06.584896
...,...,...,...,...,...,...,...,...,...
579995,2319982,19.0,10000.0,0.449744,6.0,0.5,CORR_ZX,0.9982,2025-03-07 13:58:46.447681
579996,2319986,19.0,10000.0,0.462308,6.0,0.5,CORR_ZX,1.0019,2025-03-07 13:58:54.341651
579997,2319990,19.0,10000.0,0.474872,6.0,0.5,CORR_ZX,1.0085,2025-03-07 13:59:02.711583
579998,2319994,19.0,10000.0,0.487436,6.0,0.5,CORR_ZX,1.0007,2025-03-07 13:59:10.643044


In [105]:
# df_zx.set_index('index', inplace=True)
df_zx


,d,num_shots,p,l,eta,error_type,num_log_errors,time_stamp
index,,,,,,,,
2,11.0,10000.0,0.010000,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:53.403587
6,11.0,10000.0,0.022564,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:56.653524
10,11.0,10000.0,0.035128,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:59.932352
14,11.0,10000.0,0.047692,6.0,5.0,CORR_ZX,0.0001,2024-12-05 10:38:03.298141
18,11.0,10000.0,0.060256,6.0,5.0,CORR_ZX,0.0011,2024-12-05 10:38:06.584896
...,...,...,...,...,...,...,...,...
2319982,19.0,10000.0,0.449744,6.0,0.5,CORR_ZX,0.9982,2025-03-07 13:58:46.447681
2319986,19.0,10000.0,0.462308,6.0,0.5,CORR_ZX,1.0019,2025-03-07 13:58:54.341651
2319990,19.0,10000.0,0.474872,6.0,0.5,CORR_ZX,1.0085,2025-03-07 13:59:02.711583


In [106]:
df[df['error_type'] == 'CORR_ZX'] = df_zx

In [107]:
df

,d,num_shots,p,l,eta,error_type,num_log_errors,time_stamp
0,11.0,10000.0,0.010000,6.0,5.0,X,0.0000,2024-12-05 10:37:53.398465
1,11.0,10000.0,0.010000,6.0,5.0,Z,0.0000,2024-12-05 10:37:53.402317
2,11.0,10000.0,0.010000,6.0,5.0,CORR_ZX,0.0000,2024-12-05 10:37:53.403587
3,11.0,10000.0,0.010000,6.0,5.0,TOTAL,0.0000,2024-12-05 10:37:53.404731
4,11.0,10000.0,0.022564,6.0,5.0,X,0.0000,2024-12-05 10:37:56.649020
...,...,...,...,...,...,...,...,...
2319995,19.0,10000.0,0.487436,6.0,0.5,TOTAL,1.0038,2025-03-07 13:59:10.644213
2319996,19.0,10000.0,0.500000,6.0,0.5,X,0.4986,2025-03-07 13:59:18.584272
2319997,19.0,10000.0,0.500000,6.0,0.5,Z,0.4970,2025-03-07 13:59:18.585987
2319998,19.0,10000.0,0.500000,6.0,0.5,CORR_ZX,0.9955,2025-03-07 13:59:18.587162


In [108]:
df.loc[(df['error_type'] == 'CORR_ZX')& (df['eta'] == 0.5) & (df['num_log_errors'] == (0.506 + 0.4802))]
# 13.0,10000.0,0.412051282051282,6.0,0.5,X,0.503,2025-03-07 13:42:56.470415
# 13.0,10000.0,0.412051282051282,6.0,0.5,Z,0.4802,2025-03-07 13:42:56.472368
# 13.0,10000.0,0.412051282051282,6.0,0.5,CORR_ZX,0.506,2025-03-07 13:42:56.473630

,d,num_shots,p,l,eta,error_type,num_log_errors,time_stamp
643834,17.0,10000.0,0.487436,5.0,0.5,CORR_ZX,0.9862,2025-02-12 13:16:07.045318
653906,13.0,10000.0,0.462308,5.0,0.5,CORR_ZX,0.9862,2025-02-12 13:11:25.603822
660602,17.0,10000.0,0.386923,5.0,0.5,CORR_ZX,0.9862,2025-02-12 13:16:59.205664
663962,19.0,10000.0,0.386923,5.0,0.5,CORR_ZX,0.9862,2025-02-12 13:19:41.190630
668146,11.0,10000.0,0.462308,5.0,0.5,CORR_ZX,0.9862,2025-02-12 13:08:59.358811
...,...,...,...,...,...,...,...,...
2310702,13.0,10000.0,0.449744,6.0,0.5,CORR_ZX,0.9862,2025-03-07 13:44:58.309447
2312450,15.0,10000.0,0.412051,6.0,0.5,CORR_ZX,0.9862,2025-03-07 13:44:17.726866
2316138,11.0,10000.0,0.437179,6.0,0.5,CORR_ZX,0.9862,2025-03-07 13:41:49.593052
2316610,17.0,10000.0,0.412051,6.0,0.5,CORR_ZX,0.9862,2025-03-07 13:51:19.832682


In [109]:
df.to_csv('x_corr_err_data.csv', index=False)